In [1]:
import pandas as pd
import numpy as np

import functools
import operator
pd.set_option("display.max_columns", 400)

In [2]:
file_path = "../../../data/mics6ng/raw"

# Household level data
hh = pd.read_spss(f"{file_path}/hh.sav")

# Data collected at the children's level 
ch = pd.read_spss(f"{file_path}/ch.sav")

In [31]:
ch_for_class_work = pd.read_spss("../processed_data/mics_for_class_work.sav")

In [32]:
ch_for_class_work.head()

,V1,c_mother_education,highest_grade,consent_ch,survey_ch,c_sex,c_age_month,c_age_month_cat6,attended_early_child_edu_program,health_insurance,type_of_health_insurance,has_birth_certificate,vaccination_card,received_recommended_vaccinations,ever_had_vaccination_card,c_breastfed,c_breastfeeding,c_infant_formula,c_animal_milk,c_yogurt,c_fortified_food,c_grain,c_vitA_vegs,c_roots,c_green_leafy_vegs,c_vitA_fruits,c_other_fruits_veg,c_organ_meat,c_meat,c_eggs,c_fish,c_beans_pulse,c_cheese_f_milk,DDS,DDSCategory,hh1,hh2,hhsex,hhage,helevel,hh_own_dwelling,wealth_index_score,wealth_index_hh,num_under_5_child,hh_members_num,hh_agricultural_land,hh_own_animal,fies_score,water_source,zone_hh,area_hh,hhweight,Grains_roots_and_tubers,Diary_product,Fleshy_Food,Breast_milk,Legumes_nuts,Eggs,Vitamin_A_rich_fruits_and_vegetables,Other_fruits_and_vegetables
0,0.0,4,GRADE 4,1.0,MICS,Male,39.0,36-47,1.0,0.0,,"1, SEEN","1, HAS ONLY CARD(S)",ATLEAST ONE 0 OR DK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,Male,41.0,Senior secondary,OWN,1.048265,Fourth,2.0,5.0,No,Yes,5.0,TUBE WELL / BOREHOLE,South East,URBAN,0.732652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,4,GRADE 4,1.0,MICS,Male,4.0,0-5,NaN,0.0,,"1, SEEN","1, HAS ONLY CARD(S)",,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,1.0,3.0,Male,41.0,Senior secondary,OWN,1.048265,Fourth,2.0,5.0,No,Yes,5.0,TUBE WELL / BOREHOLE,South East,URBAN,0.732652,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2.0,4,GRADE 5,1.0,MICS,Female,58.0,48-59,1.0,0.0,,"1, SEEN",,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,10.0,Male,41.0,Senior secondary,RENT,1.504502,Richest,2.0,6.0,No,No,7.0,TUBE WELL / BOREHOLE,South East,URBAN,0.732652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,4,GRADE 5,1.0,MICS,Female,3.0,0-5,NaN,0.0,,"1, SEEN","1, HAS CARD(S) AND OTHER DOCUMENT",,NaN,Yes,Yes,Yes,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,1.0,10.0,Male,41.0,Senior secondary,RENT,1.504502,Richest,2.0,6.0,No,No,7.0,TUBE WELL / BOREHOLE,South East,URBAN,0.732652,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4.0,3,GRADE 3,1.0,MICS,Male,31.0,24-35,NaN,0.0,,"1, 0T SEEN","1, HAS ONLY CARD(S)",ALL 1,NaN,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,15.0,Male,40.0,Senior secondary,RENT,1.057681,Fourth,1.0,6.0,Yes,Yes,2.0,TUBE WELL / BOREHOLE,South East,URBAN,0.732652,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [33]:
# Region # any south west, rural only
# 

In [34]:
ch_for_class_work = ch_for_class_work.drop(columns=["DDS", "DDSCategory", "V1"])

In [35]:
ch_for_class_work.zone_hh.value_counts()

zone_hh
0rth West       7743
0rth East       7508
0rth Central    6336
South West      3359
South South     3230
South East      2628
Name: count, dtype: int64

In [37]:
mask = (ch_for_class_work["zone_hh"] == "South West" ) & (ch_for_class_work["area_hh"] == "RURAL")

ch_for_class_work = ch_for_class_work[mask].reset_index(drop=True)

In [40]:
# Drop NAN in the food columns
ch_clean = ch_for_class_work.dropna(subset=["Grains_roots_and_tubers", "Diary_product", 
                                 "Fleshy_Food", "Breast_milk", "Legumes_nuts",
                                "Eggs", "Vitamin_A_rich_fruits_and_vegetables", "Other_fruits_and_vegetables"])

In [45]:
ch_clean.isna().sum()

c_mother_education                        0
highest_grade                             0
consent_ch                                0
survey_ch                                 0
c_sex                                     0
c_age_month                               0
c_age_month_cat6                          0
attended_early_child_edu_program        347
health_insurance                          2
type_of_health_insurance                  0
has_birth_certificate                     0
vaccination_card                          0
received_recommended_vaccinations         0
ever_had_vaccination_card               192
c_breastfed                               0
c_breastfeeding                           9
c_infant_formula                          1
c_animal_milk                             0
c_yogurt                                  1
c_fortified_food                          0
c_grain                                   0
c_vitA_vegs                               1
c_roots                         

In [48]:
ch_clean.to_csv("../processed_data/clean_for_classwork.csv")
ch_clean.shape

(347, 57)

In [3]:
# Turn the column names to lowercase
def rename_colnames_tolower(df):
    """Rename the column names to lower case"""
    return df.rename(str.lower, axis="columns")

In [4]:
## Household Data Analysis

In [5]:
# Turn to lower the column names
hh = rename_colnames_tolower(hh)

# household columns
hh_cols = ["hh1", "hh2", "hh6","hh7", "zone", 
           # consent
           "hh12",
           # "hh8", "
           "hh48", "hh49", "hh50", "hh51","hh52",
           "hhsex", "hhage", "helevel", "hc14"]


# Household head have and have not
hh_livestock_cols = ["hc15", "hc16", "hc17"] + [col for 
                                        col in hh.columns if col.startswith("hc18")]

# Household worried about not having enough food (FIES)
hh_fies_cols = [col for col in hh.columns if col.startswith("fe")]


# Source of water 
hh_water_cols = ["ws1", "ws2", "ws3"]

#Sanitation
hh_toilet_cols = ["ws11", "ws14", "ws15", "ws16"]


# hh wealth index columns
hh_windex_cols = ["wscore", "windex5"]


# hh survey design columns
hh_meta_data_cols = ["hhweight", "survey"]


hh_final_cols = functools.reduce(operator.add, [hh_cols, hh_windex_cols, 
                                                hh_livestock_cols, hh_water_cols, 
                                                hh_fies_cols, hh_meta_data_cols])

len(hh_final_cols)

65

In [6]:
# hh columns to rename
hh_dict = {
    "ws11": "toilet_type",
    "ws14": "toilet_location",
    "ws15": "toilet_num_hh_using",
    "ws16": "toilet_shared",
    "ws1": "water_source",
    "ws2": "other_water_source",
    "ws3": "water_source_location",
    "hh6": "area", 
    "hh7": "state",
    "hh12": "consent",
    "hc14": "hh_own_dwelling",
    "hc15": "hh_agricultural_land",
    "hc16": "hh_ag_land_size",
    "hc17": "hh_own_animal",
    "hh48": "hh_members_num",
    "hh49" : "num_of_women_15_49",
    "hh50" : "num_of_men_15_49",
    "hh51" : "num_under_5_child",
    "hh52" : "num_5_17_child", 
    # Wealth index
    "wscore": "wealth_index_score",
    "windex5": "wealth_index",

    
}

# Subset the dataframe
hh_subset = hh[hh_final_cols]
hh_subset = hh_subset.rename(columns=hh_dict)

In [7]:
# Create a boolean mask for rows where 'hh_agricultural_land' is 'NO'
mask = hh_subset['hh_agricultural_land'] == "NO"

# Replace NaN with 0 in 'hh_ag_land_size' where the mask is True
hh_subset.loc[mask, 'hh_ag_land_size'] = hh_subset.loc[mask, 'hh_ag_land_size'].fillna(0)

hh_subset.shape

(41532, 65)

In [8]:
cols_to_modify = [col for col in hh_subset.columns if col.startswith("hc18")]

mask = hh_subset["hh_own_animal"].isin(["NO", "NO RESPONSE"])
hh_subset.loc[mask, cols_to_modify] = hh_subset.loc[mask, cols_to_modify].fillna(0.0)

# Get the livestock columns and change as types from category to int. 
# Run this code onces.
hh_ls = hh_subset[cols_to_modify].apply(lambda x:x.cat.codes)

# Add a new column to indication the total livestock per hh
hh_subset["hh_ls_num"] = hh_ls.sum(axis=1)

# 
ls_cols = ["hh_own_animal", "hh1", "hh2", "hhsex", 
           "hh_own_dwelling", "hh_agricultural_land", 
           "hh_ag_land_size"] + \
        [col for col in hh_subset.columns if col.startswith("hc18")]

ls_df = hh_subset[ls_cols]

# Save data to csv
ls_df.to_csv("../processed_data/hh_livestock_dataset.csv", index=False)


# # Drop the other livestock columnes
hh_subset = hh_subset.drop(columns=cols_to_modify)

hh_subset.head()

,hh1,hh2,area,state,zone,consent,hh_members_num,num_of_women_15_49,num_of_men_15_49,num_under_5_child,num_5_17_child,hhsex,hhage,helevel,hh_own_dwelling,wealth_index_score,wealth_index,hh_agricultural_land,hh_ag_land_size,hh_own_animal,water_source,other_water_source,water_source_location,fe1,fe1a,fe1b,fe2,fe2a,fe2b,fe3,fe3a,fe3b,fe4,fe4a,fe4b,fe5,fe5a,fe5b,fe6,fe6a,fe6b,fe6c,fe7,fe7a,fe7b,fe7c,fe8,fe8a,fe8b,fe8c,hhweight,survey,hh_ls_num
0,1.0,1.0,URBAN,ABIA,South East,YES,4.0,3.0,0.0,0.0,1.0,Female,46.0,Primary,RENT,0.820389,Fourth,NO,0.0,NO,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,YES,NO,YES,YES,NO,NO,YES,NO,YES,YES,YES,YES,YES,NO,YES,YES,YES,YES,SOMETIMES (3-10 TIMES),YES,NO,YES,RARELY (1 OR 2 TIMES),NO,NaN,NaN,NaN,0.732652,MICS,0
1,1.0,2.0,URBAN,ABIA,South East,YES,2.0,2.0,0.0,0.0,1.0,Female,45.0,Senior secondary,RENT,0.700122,Fourth,NO,0.0,NO,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,YES,NO,YES,YES,NO,YES,YES,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,YES,SOMETIMES (3-10 TIMES),NO,NaN,NaN,NaN,NO,NaN,NaN,NaN,0.732652,MICS,0
2,1.0,3.0,URBAN,ABIA,South East,YES,5.0,1.0,0.0,2.0,1.0,Male,41.0,Senior secondary,OWN,1.048265,Fourth,NO,0.0,YES,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,YES,YES,NO,NO,NaN,NaN,NO,NaN,NaN,YES,YES,NO,YES,YES,NO,YES,YES,YES,RARELY (1 OR 2 TIMES),YES,YES,NO,NaN,NO,NaN,NaN,NaN,0.732652,MICS,30
3,1.0,4.0,URBAN,ABIA,South East,YES,4.0,1.0,1.0,0.0,0.0,Male,81.0,None,OWN,-0.460839,Second,YES,2.0,YES,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,NO,NaN,NaN,YES,YES,NO,YES,YES,NO,YES,YES,NO,YES,YES,NO,YES,YES,NO,NaN,YES,YES,NO,NaN,YES,YES,NO,NaN,0.732652,MICS,144
4,1.0,5.0,URBAN,ABIA,South East,YES,5.0,1.0,0.0,0.0,1.0,Male,63.0,Primary,RENT,0.983845,Fourth,NO,0.0,NO,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,SOMETIMES (3-10 TIMES),YES,YES,NO,NaN,NO,NaN,NaN,NaN,0.732652,MICS,0


In [9]:
import re

# Select primary fies question only
fies_cols = [col for col in hh_subset.columns if re.search(r'^(fe)\d+$', col)]


hh_subset[fies_cols] = hh_subset[fies_cols].apply(lambda x:x.str.upper() == "YES").astype("int8")

hh_subset = hh_subset.assign(fies_score = hh_subset[fies_cols].sum(axis=1))

# Select FIES 
# Create a list of columns including specific columns ("hh1", "hh2", "area", "zone", "hhweightmics") 
# and additional columns from hh_subset that start with "fe"
fies_cols = ["hh1", "hh2", "area", "zone","state", "hhweight"] + [col for col in hh_subset.columns if col.startswith("fe")]


# Drop the FIES columns from the hh_subset dataframe
hh_subset = hh_subset.drop(columns=hh_fies_cols, axis=1)

In [10]:
# 
ch = rename_colnames_tolower(ch)

ch_cols = ["hh1", "hh2", "ln",
           # "uf1", "uf2", "uf3",
           # mother
            "uf4",
           "melevel",
           "cinsurance",

           #
           "uf10",
           "survey",
           # sex
           "hl4",
           
           # -----ub----
           "ub2", "ub6","ub9", "ub10a",

           # ----br
           "br1", "br2","br3",

           # vaccination
           "im2",
           "im3",
           "im30",

           
           "cage","cage_6","cage_11", 
           # food groups
           "bd2", "bd3", "bd7d", "bd7e", "bd8a", "bd8b",
           "bd8c", "bd8d", "bd8e", "bd8f", "bd8g", "bd8h", 
           "bd8i", "bd8j", "bd8k", "bd8l", "bd8m", "bd8n",

           # education
           "ed5a", "ed5b",
           
           # --
           "zone", "hh6","hh7",
           #meta data
           "stratum", 
           "chweight",
           "wscore",
           "windex5"
          ]


ch_dicts = {
    "uf4" : "c_mother_ln",
    "melevel": "c_mother_edu_levl",
    "ed5a" : "highest_leve_of_edu",
    "ed5b" : "highest_grade",
    "cinsurance" : "child_health_insurance",
    
    ## ---- ub ----
    "ub2" : "c_age_yr",
    "ub6" : "attended_early_child_edu_program",
    "ub9" : "health_insurance",
    "ub10a" : "type_of_health_insurance",


    # --- br1
    "br1" : "has_birth_certificate",
    "br2" : "birth_registered",
    "br3" : "birth_registration_knowledge",

    # vaccination
    "im2": "vaccination_card",
    "im3": "ever_had_vaccination_card",
    "im30": "received_recommended_vaccinations",
    
    "uf10": "consent",
    "cage": "c_age_month",
    "cage_6" : "c_age_month_cat6",
    "cage_11" : "c_age_month_cat11",
    # -- 
    "hl4" : "c_sex",
    "melevel" : "c_mother_education",
    # Food groups
    "bd2" : "c_breastfed",
    "bd3" : "c_breastfeeding",
    "bd7d" : "c_infant_formula",
    "bd7e" : "c_animal_milk",
    "bd8a" : "c_yogurt",
    "bd8b" : "c_fortified_food",
    "bd8c" : "c_grain",
    "bd8d" : "c_vitA_vegs",
    "bd8e" : "c_roots",
    "bd8f" : "c_green_leafy_vegs",
    "bd8g" : "c_vitA_fruits",
    "bd8h" : "c_other_fruits_veg",
    "bd8i" : "c_organ_meat",
    "bd8j" : "c_meat",
    "bd8k" : "c_eggs",
    "bd8l" : "c_fish",
    "bd8m" : "c_beans_pulse",
    "bd8n" : "c_cheese_f_milk",
    
    # location
    "hh6": "area",
    "hh7": "region",
    "windex5": "wealth_index"

    # 
}


# Create a subset of the 'ch' DataFrame, including only the columns specified in 'wm_cols'
ch_subset = ch.loc[:, ch_cols]

# Rename the columns of ch_subset using the key-value pairs in ch_dicts
ch_subset = ch_subset.rename(columns=ch_dicts)

# Filter out records where consent is not "YES"
ch_subset = ch_subset[ch_subset["consent"] == "YES"]

# Filter out records where survey is not "MICS"
# ch_subset = ch_subset[ch_subset["survey"] == "MICS"].reset_index(drop=True)

In [11]:
ch_subset.head()

,hh1,hh2,ln,c_mother_ln,c_mother_education,child_health_insurance,consent,survey,c_sex,c_age_yr,attended_early_child_edu_program,health_insurance,type_of_health_insurance,has_birth_certificate,birth_registered,birth_registration_knowledge,vaccination_card,ever_had_vaccination_card,received_recommended_vaccinations,c_age_month,c_age_month_cat6,c_age_month_cat11,c_breastfed,c_breastfeeding,c_infant_formula,c_animal_milk,c_yogurt,c_fortified_food,c_grain,c_vitA_vegs,c_roots,c_green_leafy_vegs,c_vitA_fruits,c_other_fruits_veg,c_organ_meat,c_meat,c_eggs,c_fish,c_beans_pulse,c_cheese_f_milk,highest_leve_of_edu,highest_grade,zone,area,region,stratum,chweight,wscore,wealth_index
0,1.0,3.0,4.0,2.0,Higher/tertiary,Without insurance,YES,MICS,MALE,3.0,YES,NO,,"YES, SEEN",NaN,NaN,"YES, HAS ONLY CARD(S)",NaN,ATLEAST ONE NO OR DK,39.0,36-47,36-47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HIGHER/TERTIARY,GRADE 4,South East,URBAN,ABIA,1.0,0.759178,1.048265,Fourth
1,1.0,3.0,5.0,2.0,Higher/tertiary,Without insurance,YES,MICS,MALE,0.0,NaN,NO,,"YES, SEEN",NaN,NaN,"YES, HAS ONLY CARD(S)",NaN,NaN,4.0,0-5,0-11,YES,YES,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,HIGHER/TERTIARY,GRADE 4,South East,URBAN,ABIA,1.0,0.759178,1.048265,Fourth
2,1.0,10.0,5.0,2.0,Higher/tertiary,Without insurance,YES,MICS,FEMALE,4.0,YES,NO,,"YES, SEEN",NaN,NaN,NaN,NaN,NaN,58.0,48-59,48-59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HIGHER/TERTIARY,GRADE 5,South East,URBAN,ABIA,1.0,0.759178,1.504502,Richest
3,1.0,10.0,6.0,2.0,Higher/tertiary,Without insurance,YES,MICS,FEMALE,0.0,NaN,NO,,"YES, SEEN",NaN,NaN,"YES, HAS CARD(S) AND OTHER DOCUMENT",NaN,NaN,3.0,0-5,0-11,YES,YES,YES,NO,NO,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,HIGHER/TERTIARY,GRADE 5,South East,URBAN,ABIA,1.0,0.759178,1.504502,Richest
4,1.0,15.0,6.0,2.0,Senior secondary,Without insurance,YES,MICS,MALE,2.0,NaN,NO,,"YES, NOT SEEN",NaN,NaN,"YES, HAS ONLY CARD(S)",NaN,ALL YES,31.0,24-35,24-35,YES,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SENIOR SECONDARY,GRADE 3,South East,URBAN,ABIA,1.0,0.759178,1.057681,Fourth


In [12]:
hh_subset.head()

,hh1,hh2,area,state,zone,consent,hh_members_num,num_of_women_15_49,num_of_men_15_49,num_under_5_child,num_5_17_child,hhsex,hhage,helevel,hh_own_dwelling,wealth_index_score,wealth_index,hh_agricultural_land,hh_ag_land_size,hh_own_animal,water_source,other_water_source,water_source_location,hhweight,survey,hh_ls_num,fies_score
0,1.0,1.0,URBAN,ABIA,South East,YES,4.0,3.0,0.0,0.0,1.0,Female,46.0,Primary,RENT,0.820389,Fourth,NO,0.0,NO,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.732652,MICS,0,7
1,1.0,2.0,URBAN,ABIA,South East,YES,2.0,2.0,0.0,0.0,1.0,Female,45.0,Senior secondary,RENT,0.700122,Fourth,NO,0.0,NO,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.732652,MICS,0,6
2,1.0,3.0,URBAN,ABIA,South East,YES,5.0,1.0,0.0,2.0,1.0,Male,41.0,Senior secondary,OWN,1.048265,Fourth,NO,0.0,YES,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.732652,MICS,30,5
3,1.0,4.0,URBAN,ABIA,South East,YES,4.0,1.0,1.0,0.0,0.0,Male,81.0,None,OWN,-0.460839,Second,YES,2.0,YES,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.732652,MICS,144,7
4,1.0,5.0,URBAN,ABIA,South East,YES,5.0,1.0,0.0,0.0,1.0,Male,63.0,Primary,RENT,0.983845,Fourth,NO,0.0,NO,TUBE WELL / BOREHOLE,NaN,ELSEWHERE,0.732652,MICS,0,7


In [13]:
cols = ["hhsex", "hhage", "helevel", "hh_own_dwelling", "wealth_index_score",
       "wealth_index","num_under_5_child","hh_members_num", "hh_agricultural_land", 
        "hh_own_animal", "fies_score", 
        "hhweight","water_source", "survey",
       "zone", "area", "consent", "hh1", "hh2"]

hh_df = hh_subset[cols]

In [14]:
## Merge the child and household data via hh1, and hh2

In [15]:
merged_df = pd.merge(ch_subset, 
                     hh_df, 
                     on=["hh1", "hh2"], 
                     how="left",
                     suffixes=("_ch", "_hh"))
merged_df.shape
merged_df["highest_leve_of_edu"]

0         HIGHER/TERTIARY
1         HIGHER/TERTIARY
2         HIGHER/TERTIARY
3         HIGHER/TERTIARY
4        SENIOR SECONDARY
               ...       
30799                 NaN
30800                 NaN
30801                 NaN
30802                 NaN
30803                 NaN
Name: highest_leve_of_edu, Length: 30804, dtype: category
Categories (9, object): ['DK', 'ECCDE', 'HIGHER/TERTIARY', 'JUNIOR SECONDARY', ..., 'PRIMARY', 'SECONDARY TECHNICAL', 'SENIOR SECONDARY', 'VEI/IEI']

In [16]:
merged_df.head()


,hh1,hh2,ln,c_mother_ln,c_mother_education,child_health_insurance,consent_ch,survey_ch,c_sex,c_age_yr,attended_early_child_edu_program,health_insurance,type_of_health_insurance,has_birth_certificate,birth_registered,birth_registration_knowledge,vaccination_card,ever_had_vaccination_card,received_recommended_vaccinations,c_age_month,c_age_month_cat6,c_age_month_cat11,c_breastfed,c_breastfeeding,c_infant_formula,c_animal_milk,c_yogurt,c_fortified_food,c_grain,c_vitA_vegs,c_roots,c_green_leafy_vegs,c_vitA_fruits,c_other_fruits_veg,c_organ_meat,c_meat,c_eggs,c_fish,c_beans_pulse,c_cheese_f_milk,highest_leve_of_edu,highest_grade,zone_ch,area_ch,region,stratum,chweight,wscore,wealth_index_ch,hhsex,hhage,helevel,hh_own_dwelling,wealth_index_score,wealth_index_hh,num_under_5_child,hh_members_num,hh_agricultural_land,hh_own_animal,fies_score,hhweight,water_source,survey_hh,zone_hh,area_hh,consent_hh
0,1.0,3.0,4.0,2.0,Higher/tertiary,Without insurance,YES,MICS,MALE,3.0,YES,NO,,"YES, SEEN",NaN,NaN,"YES, HAS ONLY CARD(S)",NaN,ATLEAST ONE NO OR DK,39.0,36-47,36-47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HIGHER/TERTIARY,GRADE 4,South East,URBAN,ABIA,1.0,0.759178,1.048265,Fourth,Male,41.0,Senior secondary,OWN,1.048265,Fourth,2.0,5.0,NO,YES,5,0.732652,TUBE WELL / BOREHOLE,MICS,South East,URBAN,YES
1,1.0,3.0,5.0,2.0,Higher/tertiary,Without insurance,YES,MICS,MALE,0.0,NaN,NO,,"YES, SEEN",NaN,NaN,"YES, HAS ONLY CARD(S)",NaN,NaN,4.0,0-5,0-11,YES,YES,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,HIGHER/TERTIARY,GRADE 4,South East,URBAN,ABIA,1.0,0.759178,1.048265,Fourth,Male,41.0,Senior secondary,OWN,1.048265,Fourth,2.0,5.0,NO,YES,5,0.732652,TUBE WELL / BOREHOLE,MICS,South East,URBAN,YES
2,1.0,10.0,5.0,2.0,Higher/tertiary,Without insurance,YES,MICS,FEMALE,4.0,YES,NO,,"YES, SEEN",NaN,NaN,NaN,NaN,NaN,58.0,48-59,48-59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HIGHER/TERTIARY,GRADE 5,South East,URBAN,ABIA,1.0,0.759178,1.504502,Richest,Male,41.0,Senior secondary,RENT,1.504502,Richest,2.0,6.0,NO,NO,7,0.732652,TUBE WELL / BOREHOLE,MICS,South East,URBAN,YES
3,1.0,10.0,6.0,2.0,Higher/tertiary,Without insurance,YES,MICS,FEMALE,0.0,NaN,NO,,"YES, SEEN",NaN,NaN,"YES, HAS CARD(S) AND OTHER DOCUMENT",NaN,NaN,3.0,0-5,0-11,YES,YES,YES,NO,NO,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,HIGHER/TERTIARY,GRADE 5,South East,URBAN,ABIA,1.0,0.759178,1.504502,Richest,Male,41.0,Senior secondary,RENT,1.504502,Richest,2.0,6.0,NO,NO,7,0.732652,TUBE WELL / BOREHOLE,MICS,South East,URBAN,YES
4,1.0,15.0,6.0,2.0,Senior secondary,Without insurance,YES,MICS,MALE,2.0,NaN,NO,,"YES, NOT SEEN",NaN,NaN,"YES, HAS ONLY CARD(S)",NaN,ALL YES,31.0,24-35,24-35,YES,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SENIOR SECONDARY,GRADE 3,South East,URBAN,ABIA,1.0,0.759178,1.057681,Fourth,Male,40.0,Senior secondary,RENT,1.057681,Fourth,1.0,6.0,YES,YES,2,0.732652,TUBE WELL / BOREHOLE,MICS,South East,URBAN,YES


In [17]:
colofinterest_hh = ["hh1",
                    "hh2",
                    "hhsex", 
                    "hhage", 
                    "helevel", 
                    "hh_own_dwelling", 
                    "wealth_index_score",
                    "wealth_index_hh",
                    "num_under_5_child",
                    "hh_members_num", 
                    "hh_agricultural_land",
                    "hh_own_animal", 
                    "fies_score", 
                    "water_source",
                    "zone_hh", 
                    "area_hh",
                    "hhweight"
                   ]

colofinterest_ch = ["c_mother_education", 
                    # "highest_leve_of_edu", 
                    "highest_grade",
                    "consent_ch", 
                    "survey_ch", 
                    "c_sex",
                    "c_age_month",
                    "c_age_month_cat6",
                    "attended_early_child_edu_program",
                    "health_insurance", 
                    "type_of_health_insurance", 
                    "has_birth_certificate",
                    "vaccination_card", 
                    "received_recommended_vaccinations", 
                    "ever_had_vaccination_card",
                    "c_breastfed",
                    "c_breastfeeding",
                    "c_infant_formula",
                    "c_animal_milk",
                    "c_yogurt",
                    "c_fortified_food",
                    "c_grain",
                    "c_vitA_vegs",
                    "c_roots",
                    "c_green_leafy_vegs",
                    "c_vitA_fruits",
                    "c_other_fruits_veg",
                    "c_organ_meat",
                    "c_meat",
                    "c_eggs",
                    "c_fish",
                    "c_beans_pulse",
                    "c_cheese_f_milk",
                    ]

cols = colofinterest_ch+colofinterest_hh

In [18]:
ch_merged_hh = merged_df[cols]
ch_merged_hh.to_csv("../processed_data/hh_plus_children_data.csv")

In [19]:
ch_merged_hh.columns.unique()

Index(['c_mother_education', 'highest_grade', 'consent_ch', 'survey_ch',
       'c_sex', 'c_age_month', 'c_age_month_cat6',
       'attended_early_child_edu_program', 'health_insurance',
       'type_of_health_insurance', 'has_birth_certificate', 'vaccination_card',
       'received_recommended_vaccinations', 'ever_had_vaccination_card',
       'c_breastfed', 'c_breastfeeding', 'c_infant_formula', 'c_animal_milk',
       'c_yogurt', 'c_fortified_food', 'c_grain', 'c_vitA_vegs', 'c_roots',
       'c_green_leafy_vegs', 'c_vitA_fruits', 'c_other_fruits_veg',
       'c_organ_meat', 'c_meat', 'c_eggs', 'c_fish', 'c_beans_pulse',
       'c_cheese_f_milk', 'hh1', 'hh2', 'hhsex', 'hhage', 'helevel',
       'hh_own_dwelling', 'wealth_index_score', 'wealth_index_hh',
       'num_under_5_child', 'hh_members_num', 'hh_agricultural_land',
       'hh_own_animal', 'fies_score', 'water_source', 'zone_hh', 'area_hh',
       'hhweight'],
      dtype='object')

In [26]:
ch_merged_hh.c_mother_education.value_counts()

c_mother_education
None                12944
Senior secondary     7998
Primary              5036
Higher/tertiary      2573
Junior secondary     2249
Missing/DK              4
Name: count, dtype: int64